In [26]:
import numpy as np
import pandas as pd

# Data 전처리

In [27]:
# csv를 읽는다. 
#header = userId , movieId, rating. timestamp
raw_data = np.array(pd.read_csv('./MovieLens/ratings.csv'))
raw_data.shape

(100004, 4)

In [28]:
# 유저 몇명있는지 (행) 영화가 몇개있는지(열)를 찾는다. 
#1. 구별되는 유저아이디와, 영화아이디를 찾는다.
userId = np.unique(raw_data.T[0]) 
movieIdPos = np.unique(raw_data.T[1])
#2. 각각의 사이즈를 구함으로써, 유저수와 영화의 갯수를 알아낼수 있다. 
N = userId.size
M = movieIdPos.size

#이걸이용해서 모든 칸이 0인 유저*무비 매트릭스를 만든다.(기존 예제의 R) = ratings
ratings = np.zeros((N,M))
ratings.shape


(671, 9066)

In [29]:
print movieIdPos[M-1] 

163949.0


In [30]:
#영화의 갯수는 9066개 인데, 마지막영화의 아이디가 163949이다. 이 차이를 해소하기 위한 함수를 별도로 구현하였다. 
def get_position(movieId) :
    mid = np.where(movieIdPos == movieId)
    np.asanyarray(mid)
    mid = mid[0]
    movieIdPosition = mid[0]
   
    return movieIdPosition

In [31]:
#raw_data를 이용하여, ratings 매트릭스 완성한다. 
#1. 가지고있는 유저-영화 별 평점의갯수 = 몇회의 iterating?
iteration = raw_data.shape[0] 

#2. 매트릭스에 저장. 
for i in range (iteration):
    uid = raw_data[i][0] -1 #행렬이 0부터 시작하므로 1번유저는 0번줄에 저장 
    mid_pos = raw_data[i][1]
    mid = get_position(mid_pos)
    rating = raw_data[i][2]

    ratings[uid][mid] = rating



/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [32]:
print ratings

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 4.  0.  0. ...,  0.  0.  0.]
 [ 5.  0.  0. ...,  0.  0.  0.]]


# MF 구현부 (기존 코드와 똑같음)

In [24]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        print step
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
#                    for k in xrange(K):
                    P[i,:] = P[i,:] + alpha * (2 * eij * Q[:,j] - beta * P[i,:])
                    Q[:,j] = Q[:,j] + alpha * (2 * eij * P[i,:] - beta * Q[:,j])    
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T

In [25]:
#변수 선언
R = ratings
K = 2 
#P = np.random.rand(N,K)
#Q = np.random.rand(M,K)

P = np.random.uniform(low=0, high=5,size=(N,K))
print "P : \n", P.shape
print P

np.random.seed(2)
#Q = np.random.rand(M,K) # 유저의수 
Q = np.random.uniform(low=0, high=5,size=(M,K))
print "Q : \n", Q.shape
print Q
#연산
nP, nQ = matrix_factorization(R, P, Q, K)


P : 
(671, 2)
[[ 1.43167279  0.3091497 ]
 [ 1.13000988  2.31642468]
 [ 2.56574779  0.9851403 ]
 ..., 
 [ 3.19999206  2.21082233]
 [ 2.91394866  3.24287104]
 [ 1.03207787  0.15940389]]
Q : 
(9066, 2)
[[ 2.17997451  0.12963116]
 [ 2.74831239  2.17661196]
 [ 2.10183901  1.65167411]
 ..., 
 [ 1.49191858  1.13586114]
 [ 1.82450218  4.26354078]
 [ 2.64728723  0.25582542]]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


KeyboardInterrupt: 

In [ ]:
nR = np.dot(nP, nQ.T)
print nR